# The Bancor Pricing Algorithm
Based on the whitepaper from: Eyal Hertzog, Guy Benartzi, Galia Benartzi, "The Bancor Protocol" March, 2018

by: gunnar pope on 8/22/18




### Definitions and notes on the Bancor Algorithm:

Use the definition of $TOK$ for the Smart Token symbol and $CTOK$ for the connector token that you wish to transfer.

#### Connector Weight
From the Price Formula algorithms:

$$ CW = \frac{ConnectorBalance}{SmartTokenTotalValue}$$
Where, $ConnectorBalance$ is the balance of the toconnector tokens in the connector account and $SmartTokenTotalValue$ is the market cap of the Smart token implementing the Bancor algorithm.

$$SmartTokenTotalValue= price*TotalTOKSupply$$



Where $price$ is in units of [USD/TOK] and $SmartTokenSupply$ is the maximum amount of $TOK$ tokens.

#### Token Price
Solve for the Smart Token's price as a function of it's $CW$, $ConnectorBalance$, and $OutstandingTOKSupply$.  The $OutstandingTOKSupply$ is not defined in the Bancor whitepaper, but I'm assuming it represents the difference between the total circulating supply and the total available supply of Smart Tokens. This $OutstandingSupply$ would have to be a liquid balance of $TOK$, owned by the Smart Contract administrators, to be usefully traded in a smart contract.

$$OutstandingTOKSupply = TotalTOKSupply - CirculatingTOKSupply$$


The Whitepaper defines an algorithm for determining the price of $TOK$ by:

$$ price = \frac{CTOKBalance}{OutstandingTOKSupply*CW}$$

This formula must be in units of $[CTOK/TOK]$, and is uses $CW$ to scale the price ratio. CTOKbalance is $[CTOK]$ and $OutstandingTOKSupply$ is in $[TOK]$.


This formula implies that if there is a large outstanding supply (low demand), the token will be worth a fraction of it's marketcap price and if only a few tokens are available in supply (high demand), each token will be worth nearly the marketcap price.


#### Tokens Issued 


$$ TOKissued = TOKsupply*\big((1+(\frac{CTOKpaid}{CTOKbalance})^{CW}-1\big)$$

This is the amount of Smart Tokens to payout given the current smart token supply, $TOKsupply$, the connected tokens paid, $CTOKpaid$, and the balance of $CTOK$ tokens in the bancor account, $CTOKbalance$.


$$ CTOKpaidout = CTOKbalance * ((1+\frac{TOKdestroyed}{TOKsupply})^{\frac{1}{CW}}-1) $$

The effective price can be calculated as:

$$ effectivePrice  [CTOK/TOK] = \frac{CTOKexchanged}{TOKexchanged}$$

The $effectivePrice$ represents the effective exchange rate of the transaction, in units of $[CTOK/TOK]$




In [14]:
# Initializations

# HORUSTotalsupply  = 1000000
# HORUScurrentsupply= 1000
# HORUSprice        = 0.02 #USD
# HorusMarketCap    = HORUSprice * HORUScurrentsupply
# EOSbalance        = 10000
HORUSTotalsupply  = 1000
HORUScurrentsupply= 1000
HORUSprice        = 0.02 #USD
HorusMarketCap    = HORUSprice * HORUScurrentsupply
EOSbalance        = 0.5*HorusMarketCap

connector_balance = EOSbalance
SmartTokenTotalValue = HorusMarketCap
SmartTokenSupply = HORUScurrentsupply

class smart_token:
    def __init__(self, name, circulatingsupply, totalsupply , price):
        self.name = name
        self.circulatingsupply = circulatingsupply
        self.totalsupply = totalsupply
        self.price = price                   # USD/HORUS
        self.marketcap = price * totalsupply # USD/HORUS*TOKsupply
        self.outstandingsupply = self.totalsupply - self.circulatingsupply
        
    def get_marketcap(self):
        return (self.price * self.totalsupply)
    
    def get_outstandingsupply(self):
        return(self.totalsuply - self.circulatingsupply)
    
    
class connector:
    def __init__(self, tokenname, balance, price):
        self.token   = tokenname   # name="TOK", for example
        self.balance = balance     # the balance of token "TOK" remaining
        self.price   = price       # USD/TOK
        self.balanceValue = self.price*self.balance
        self.CTOKpaid = 0
        
    def get_balanceValue(self):
         return (self.price * self.balance)
        
    # CW = connectorBalanceValue/SmartTokenTotalValue
    def get_CW(self, TOKMarketCap):
        CW = self.balance / TOKMarketCap
        return (CW)

class Bancor:
    def __init__(self):
        price = 0
        TOKissued = 0
        CTOKpaidout = 0
        effectiveprice = 0
        
    def priceCTOKtoTOK(CTOKbalance, TOKoutsupply, CW):
        self.price = CTOKbalance/(TOKoutsupply*CW)
        return ( self.price )
    
    def TOKissued(TOKsupply, CTOKpaid, CTOKbalance,CW):
        self.TOKissued = TOKsupply*( (1+CTOKpaid/CTOKbalance)**(CW) -1)
        return (self.TOKissued)
    def CTOKpaidout(TOKdestroyed,TOKsupply,CW,CTOKbalance):
        CTOKpaidout = CTOKbalance*( (1+ TOKdestroyed/TOKsupply)**(1/(CW)) -1)
        return (CTOKpaidout)
    def effectiveCTOKtoTOK(CTOKexchanged, TOKexchanged):
        return (CTOKexchanged/TOKexchanged)
                                   
                                
        
###########################################


# eoscon.price = 1
# eoscon.get_balanceValue()

# find the supply of HORUS on the EOS MAINNET
# $ cleos --url https://api.eosnewyork.io:443  get currency stats horustokenio "HORUS"
# {
#   "HORUS": {
#     "supply": "1200000000.0000 HORUS",
#     "max_supply": "1200000000.0000 HORUS",
#     "issuer": "horustokenio"
#   }


# on 8/22/18, the current HORUS price is:
USDtoHORUS  = 0.0031 # in units of USD/HORUS
print("USDtoHorus", USDtoHORUS)


# Initialize your smart token, (use an estimated circulation of 10000 HORUS)
horus = smart_token("HORUS", 10000.0000, 1200000000.0000, USDtoHORUS)

eoscon = connector("EOS",EOSbalance,4.3)
eoscon.balance = 0.5* horus.marketcap# explicitly set this for testing

bancor = Bancor()

print("Horus Market Cap", horus.marketcap)

# The connector weight
CW = eoscon.get_CW(horus.marketcap)
print("The Connector Weight: ", CW)

# The tokens issued
eoscon.CTOKpaid = 10 # the user wants to exchange 10 EOS tokens for horus

# TOKissued = bancor.TOKissued(horus.totalsupply, eoscon.CTOKpaid, eoscon.balance, CW)
# print("The Tokens Issued: ", TOKissued)

# bancor pricing
# print("Get the Price", get_price_CTOKtoTOK(horus.outstandingsupply, horus.totalsupply, horus.price))
# price = connector_balance / (SmartTokenSupply * CW)
                                   
# The exchange price
# print("The Exchange Price: ", )
# CW = connectorBalanceValue/SmartTokenTotalValue

# bancor pricing
# price = connector_balance / (SmartTokenSupply * CW)

# # tokens issued
# # these are wrong and need to be edited!
# tokensIssued = SmartTokenSupply * ( (1 + tokensDestroyed/SmartTokenSupply)**(1/CW) -1)


# tokensIssued = SmartTokenSupply * ( (1 + tokensDestroyed/SmartTokenSupply)**(1/CW) -1)




USDtoHorus 0.0031
Horus Market Cap 3720000.0
The Connector Weight:  0.5


In [6]:
class connector:
    def __init__(self, tokenname, balance, price):
        self.token   = tokenname
        self.balance = balance
        self.price   = price # $TOKEN/USD
        self.balanceValue = self.price*self.balance
        
    def get_balanceValue(self):
         return (self.price * self.balance)
        
EOScon = connector("EOS",1000,0.1)
EOScon.balanceValue
EOScon.price = 1
EOScon.get_balanceValue()

1000